<a href="https://colab.research.google.com/github/gitgubbenito/capstone/blob/main/VAERS-v2.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##Capstone Project - Vaccine Adverse Event Reporting System

Data files can be found at the following location:
https://vaers.hhs.gov/data/datasets.html

Datasets were to big to be uploaded to github

I focus on 2021 datasets

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#pip install chardet

In [ ]:
import chardet
with open('/content/drive/MyDrive/capstone/2021VAERSSYMPTOMS.csv', 'rb') as f:
    data = f.read()
    encoding_result = chardet.detect(data)
    encoding = encoding_result['encoding']
    print("Detected Encoding:", encoding)

Detected Encoding: ascii


#Things to do
- Load all 2021 files - done
- Understand Datasets - done
- Merge Datasets - done
- Data Clean-up - done
- Determine the most common sysmtoms by vaccine manu
- How many people were disable o die due to the vaccine
- Number of VAERS by state
- Plot top 20 symtoms
# Questions to be answer
1.  Do people who experience side effects from the vaccine tend to experience them sooner or later after vaccination?
2.  Can factors like age, gender, vaccine brand, and medical conditions help predict how strongly someone might react to the vaccine?
3.  Is there a noticeable difference in age between those who have no serious side effects and those who do?
4. What are the most common VAERS presented on pacients with COVID-19 vaccine






In [ ]:
# load data sets.csv
df_symptoms_2021 =pd.read_csv('/content/drive/MyDrive/capstone/2021VAERSSYMPTOMS.csv')
df_vaers_data_2021 = pd.read_csv('/content/drive/MyDrive/capstone/2021VAERSDATA.csv',encoding="Windows-1252")
df_vaers_vax_2021 = pd.read_csv('/content/drive/MyDrive/capstone/2021VAERSVAX.csv',encoding="Windows-1252")


<ipython-input-29-1dccfd9fb363>:3: DtypeWarning: Columns (7,12,15,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df_vaers_data_2021 = pd.read_csv('/content/drive/MyDrive/capstone/2021VAERSDATA.csv',encoding="Windows-1252")


In [ ]:
# Shape for each dataframe

print("Shape of df_symptoms_2021:", df_symptoms_2021.shape)
print("Shape of df_vaers_data_2021:", df_vaers_data_2021.shape)
print("Shape of df_vaers_vax_2021:", df_vaers_vax_2021.shape)


Shape of df_symptoms_2021: (1003896, 11)
Shape of df_vaers_data_2021: (753006, 35)
Shape of df_vaers_vax_2021: (797081, 8)


In [ ]:
df_vaers_vax_2021.isnull().sum()

VAERS_ID                0
VAX_TYPE                0
VAX_MANU                0
VAX_LOT            244925
VAX_DOSE_SERIES      4169
VAX_ROUTE          190149
VAX_SITE           220146
VAX_NAME                0
dtype: int64

In [ ]:
df_vaers_data_2021.isnull().sum()

VAERS_ID             0
RECVDATE             0
STATE           115887
AGE_YRS          81007
CAGE_YR         148654
CAGE_MO         748705
SEX                  0
RPT_DATE        752078
SYMPTOM_TEXT       577
DIED            742456
DATEDIED        743627
L_THREAT        741891
ER_VISIT        752879
HOSPITAL        705537
HOSPDAYS        721807
X_STAY          752628
DISABLE         741033
RECOVD           73431
VAX_DATE         55745
ONSET_DATE       65815
NUMDAYS          94529
LAB_DATA        523797
V_ADMINBY            0
V_FUNDBY        752009
OTHER_MEDS      380890
CUR_ILL         544477
HISTORY         378107
PRIOR_VAX       716552
SPLTTYPE        533091
FORM_VERS            0
TODAYS_DATE       5560
BIRTH_DEFECT    752547
OFC_VISIT       608478
ER_ED_VISIT     662703
ALLERGIES       454461
dtype: int64

In [ ]:
# Drop  Columns
clean_vaers_data_df = df_vaers_data_2021.drop(columns =['CAGE_YR', 'CAGE_MO', 'RPT_DATE', 'ONSET_DATE','NUMDAYS', 'LAB_DATA', 'SYMPTOM_TEXT',
                                                   'V_ADMINBY', 'DATEDIED', 'V_FUNDBY', 'OTHER_MEDS', 'CUR_ILL', 'PRIOR_VAX','VAX_DATE',
                                                   'SPLTTYPE', 'FORM_VERS', 'TODAYS_DATE', 'OFC_VISIT', 'HISTORY', 'ER_ED_VISIT', 'ALLERGIES',
                                                   'RECVDATE','STATE','HOSPDAYS','X_STAY','RECOVD'])

In [ ]:
clean_vaers_data_df.columns

Index(['VAERS_ID', 'AGE_YRS', 'SEX', 'DIED', 'L_THREAT', 'ER_VISIT',
       'HOSPITAL', 'DISABLE', 'BIRTH_DEFECT'],
      dtype='object')

In [ ]:
df_symptoms_2021.isnull().sum()

VAERS_ID                0
SYMPTOM1                0
SYMPTOMVERSION1         0
SYMPTOM2           225538
SYMPTOMVERSION2    225538
SYMPTOM3           408255
SYMPTOMVERSION3    408255
SYMPTOM4           551438
SYMPTOMVERSION4    551438
SYMPTOM5           665894
SYMPTOMVERSION5    665894
dtype: int64

**Note:** I will focus in column: SYMPTOM1

In [ ]:
df_vaers_vax_2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 797081 entries, 0 to 797080
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   VAERS_ID         797081 non-null  int64 
 1   VAX_TYPE         797081 non-null  object
 2   VAX_MANU         797081 non-null  object
 3   VAX_LOT          552156 non-null  object
 4   VAX_DOSE_SERIES  792912 non-null  object
 5   VAX_ROUTE        606932 non-null  object
 6   VAX_SITE         576935 non-null  object
 7   VAX_NAME         797081 non-null  object
dtypes: int64(1), object(7)
memory usage: 48.7+ MB


In [ ]:
df_symptoms_2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1003896 entries, 0 to 1003895
Data columns (total 11 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   VAERS_ID         1003896 non-null  int64  
 1   SYMPTOM1         1003896 non-null  object 
 2   SYMPTOMVERSION1  1003896 non-null  float64
 3   SYMPTOM2         778358 non-null   object 
 4   SYMPTOMVERSION2  778358 non-null   float64
 5   SYMPTOM3         595641 non-null   object 
 6   SYMPTOMVERSION3  595641 non-null   float64
 7   SYMPTOM4         452458 non-null   object 
 8   SYMPTOMVERSION4  452458 non-null   float64
 9   SYMPTOM5         338002 non-null   object 
 10  SYMPTOMVERSION5  338002 non-null   float64
dtypes: float64(5), int64(1), object(5)
memory usage: 84.3+ MB


In [ ]:
# prompt: how many vaers are by vacine manu

df_vaers_vax_2021['VAX_MANU'].value_counts()


VAX_MANU
MODERNA                              349254
PFIZER\BIONTECH                      328949
JANSSEN                               62327
UNKNOWN MANUFACTURER                  16709
GLAXOSMITHKLINE BIOLOGICALS           16291
MERCK & CO. INC.                      10490
SANOFI PASTEUR                         6888
SEQIRUS, INC.                          2155
NOVARTIS VACCINES AND DIAGNOSTICS      1474
PFIZER\WYETH                           1306
PROTEIN SCIENCES CORPORATION            535
DYNAVAX TECHNOLOGIES CORPORATION        247
EMERGENT BIOSOLUTIONS                   137
BERNA BIOTECH, LTD.                      97
MEDIMMUNE VACCINES, INC.                 68
TEVA PHARMACEUTICALS                     43
MASS. PUB HLTH BIOL LAB                  29
PAXVAX                                   25
MSP VACCINE COMPANY                      22
INTERCELL AG                             17
CSL LIMITED                               9
SMITHKLINE BEECHAM                        7
PASTEUR MERIEUX CONNAUG

In [ ]:
# prompt: merge all 3 data frame

df_combine_2021 = pd.merge(df_vaers_data_2021, df_vaers_vax_2021, on='VAERS_ID', how='inner')
df_combine_2021 = pd.merge(df_combine_2021, df_symptoms_2021, on='VAERS_ID', how='inner')


In [ ]:
df_combine_2021.shape

(1070234, 52)

In [ ]:
df_combine_2021.columns

Index(['VAERS_ID', 'RECVDATE', 'STATE', 'AGE_YRS', 'CAGE_YR', 'CAGE_MO', 'SEX',
       'RPT_DATE', 'SYMPTOM_TEXT', 'DIED', 'DATEDIED', 'L_THREAT', 'ER_VISIT',
       'HOSPITAL', 'HOSPDAYS', 'X_STAY', 'DISABLE', 'RECOVD', 'VAX_DATE',
       'ONSET_DATE', 'NUMDAYS', 'LAB_DATA', 'V_ADMINBY', 'V_FUNDBY',
       'OTHER_MEDS', 'CUR_ILL', 'HISTORY', 'PRIOR_VAX', 'SPLTTYPE',
       'FORM_VERS', 'TODAYS_DATE', 'BIRTH_DEFECT', 'OFC_VISIT', 'ER_ED_VISIT',
       'ALLERGIES', 'VAX_TYPE', 'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES',
       'VAX_ROUTE', 'VAX_SITE', 'VAX_NAME', 'SYMPTOM1', 'SYMPTOMVERSION1',
       'SYMPTOM2', 'SYMPTOMVERSION2', 'SYMPTOM3', 'SYMPTOMVERSION3',
       'SYMPTOM4', 'SYMPTOMVERSION4', 'SYMPTOM5', 'SYMPTOMVERSION5'],
      dtype='object')

In [ ]:
# How many people die by vacine manu order by count

df_combine_2021.groupby('VAX_MANU')['DIED'].value_counts().sort_values(ascending=False)


VAX_MANU                           DIED
PFIZER\BIONTECH                    Y       9646
MODERNA                            Y       8160
JANSSEN                            Y       1901
UNKNOWN MANUFACTURER               Y        374
MERCK & CO. INC.                   Y         92
GLAXOSMITHKLINE BIOLOGICALS        Y         78
SANOFI PASTEUR                     Y         77
PFIZER\WYETH                       Y         39
SEQIRUS, INC.                      Y         14
PROTEIN SCIENCES CORPORATION       Y          4
NOVARTIS VACCINES AND DIAGNOSTICS  Y          3
DYNAVAX TECHNOLOGIES CORPORATION   Y          2
PAXVAX                             Y          2
Name: count, dtype: int64

In [ ]:
# prompt: how many rows and columns are there for each df

print("df_symptoms_2021 has", df_symptoms_2021.shape[0], "rows and", df_symptoms_2021.shape[1], "columns")
print("df_vaers_data_2021 has", df_vaers_data_2021.shape[0], "rows and", df_vaers_data_2021.shape[1], "columns")
print("df_vaers_vax_2021 has", df_vaers_vax_2021.shape[0], "rows and", df_vaers_vax_2021.shape[1], "columns")


df_symptoms_2021 has 1003896 rows and 11 columns
df_vaers_data_2021 has 753006 rows and 35 columns
df_vaers_vax_2021 has 797081 rows and 8 columns


In [ ]:
df_combine_2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1070234 entries, 0 to 1070233
Data columns (total 47 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   VAERS_ID         1070234 non-null  int64  
 1   RECVDATE         1070234 non-null  object 
 2   STATE            923390 non-null   object 
 3   AGE_YRS          975545 non-null   float64
 4   CAGE_YR          881058 non-null   float64
 5   CAGE_MO          8245 non-null     float64
 6   SEX              1070234 non-null  object 
 7   RPT_DATE         1293 non-null     object 
 8   SYMPTOM_TEXT     1069604 non-null  object 
 9   DIED             20392 non-null    object 
 10  DATEDIED         18847 non-null    object 
 11  L_THREAT         26261 non-null    object 
 12  ER_VISIT         192 non-null      object 
 13  HOSPITAL         112784 non-null   object 
 14  HOSPDAYS         80015 non-null    float64
 15  X_STAY           771 non-null      object 
 16  DISABLE          2

In [ ]:
df_combine_2021.head()

,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,...,SYMPTOM1,SYMPTOMVERSION1,SYMPTOM2,SYMPTOMVERSION2,SYMPTOM3,SYMPTOMVERSION3,SYMPTOM4,SYMPTOMVERSION4,SYMPTOM5,SYMPTOMVERSION5
0,916600,01/01/2021,TX,33.0,33.0,NaN,F,NaN,Right side of epiglottis swelled up and hinder...,NaN,...,Dysphagia,23.1,Epiglottitis,23.1,NaN,NaN,NaN,NaN,NaN,NaN
1,916601,01/01/2021,CA,73.0,73.0,NaN,F,NaN,Approximately 30 min post vaccination administ...,NaN,...,Anxiety,23.1,Dyspnoea,23.1,NaN,NaN,NaN,NaN,NaN,NaN
2,916602,01/01/2021,WA,23.0,23.0,NaN,F,NaN,"About 15 minutes after receiving the vaccine, ...",NaN,...,Chest discomfort,23.1,Dysphagia,23.1,Pain in extremity,23.1,Visual impairment,23.1,NaN,NaN
3,916603,01/01/2021,WA,58.0,58.0,NaN,F,NaN,"extreme fatigue, dizziness,. could not lift my...",NaN,...,Dizziness,23.1,Fatigue,23.1,Mobility decreased,23.1,NaN,NaN,NaN,NaN
4,916604,01/01/2021,TX,47.0,47.0,NaN,F,NaN,"Injection site swelling, redness, warm to the ...",NaN,...,Injection site erythema,23.1,Injection site pruritus,23.1,Injection site swelling,23.1,Injection site warmth,23.1,NaN,NaN


In [ ]:
df_combine_2021.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334491 entries, 0 to 334490
Data columns (total 52 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   VAERS_ID         334491 non-null  int64  
 1   RECVDATE         334491 non-null  object 
 2   STATE            285041 non-null  object 
 3   AGE_YRS          318276 non-null  float64
 4   CAGE_YR          302536 non-null  float64
 5   CAGE_MO          2625 non-null    float64
 6   SEX              334491 non-null  object 
 7   RPT_DATE         701 non-null     object 
 8   SYMPTOM_TEXT     334341 non-null  object 
 9   DIED             4527 non-null    object 
 10  DATEDIED         4204 non-null    object 
 11  L_THREAT         5010 non-null    object 
 12  ER_VISIT         83 non-null      object 
 13  HOSPITAL         16957 non-null   object 
 14  HOSPDAYS         12072 non-null   float64
 15  X_STAY           183 non-null     object 
 16  DISABLE          3725 non-null    obje

In [ ]:
# prompt: Determine the top 12 most common sysmtoms by vaccine manu

top_12_symptoms = df_combine_2021.groupby('VAX_MANU')['SYMPTOM1'].value_counts().sort_values(ascending=False).head(12)
print(top_12_symptoms)


VAX_MANU         SYMPTOM1               
MODERNA          Chills                     28733
                 Arthralgia                 20233
PFIZER\BIONTECH  Chills                     19162
                 Arthralgia                 17683
                 COVID-19                   17323
                 Dizziness                  16167
MODERNA          Injection site erythema    14783
                 Fatigue                    13872
                 Headache                   12506
PFIZER\BIONTECH  Fatigue                    11969
MODERNA          Dizziness                  11868
PFIZER\BIONTECH  Headache                   10987
Name: count, dtype: int64


In [ ]:
# prompt: how many people die by vacine manu

#df_combine_2021.groupby('VAX_MANU')['DIED'].sum()
df_combine_2021.groupby('VAX_MANU')['DIED'].value_counts()


VAX_MANU                           DIED
DYNAVAX TECHNOLOGIES CORPORATION   Y          1
GLAXOSMITHKLINE BIOLOGICALS        Y         20
JANSSEN                            Y        117
MERCK & CO. INC.                   Y         20
MODERNA                            Y       2136
NOVARTIS VACCINES AND DIAGNOSTICS  Y          1
PFIZER\BIONTECH                    Y       2090
PFIZER\WYETH                       Y          8
PROTEIN SCIENCES CORPORATION       Y          4
SANOFI PASTEUR                     Y         19
UNKNOWN MANUFACTURER               Y        111
Name: count, dtype: int64

In [ ]:
# prompt: set the index as VAERS_ID

df_symptoms = df_combine_2021.set_index('VAERS_ID')


In [ ]:
df_symptoms.head(10)

,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,DATEDIED,...,VAX_LOT,VAX_DOSE_SERIES,VAX_ROUTE,VAX_SITE,VAX_NAME,SYMPTOM1,SYMPTOM2,SYMPTOM3,SYMPTOM4,SYMPTOM5
VAERS_ID,,,,,,,,,,,,,,,,,,,,,
916600,01/01/2021,TX,33.0,33.0,NaN,F,NaN,Right side of epiglottis swelled up and hinder...,NaN,NaN,...,037K20A,1,IM,LA,COVID19 (COVID19 (MODERNA)),Dysphagia,Epiglottitis,NaN,NaN,NaN
916601,01/01/2021,CA,73.0,73.0,NaN,F,NaN,Approximately 30 min post vaccination administ...,NaN,NaN,...,025L20A,1,IM,RA,COVID19 (COVID19 (MODERNA)),Anxiety,Dyspnoea,NaN,NaN,NaN
916602,01/01/2021,WA,23.0,23.0,NaN,F,NaN,"About 15 minutes after receiving the vaccine, ...",NaN,NaN,...,EL1284,1,IM,LA,COVID19 (COVID19 (PFIZER-BIONTECH)),Chest discomfort,Dysphagia,Pain in extremity,Visual impairment,NaN
916603,01/01/2021,WA,58.0,58.0,NaN,F,NaN,"extreme fatigue, dizziness,. could not lift my...",NaN,NaN,...,unknown,UNK,NaN,NaN,COVID19 (COVID19 (MODERNA)),Dizziness,Fatigue,Mobility decreased,NaN,NaN
916604,01/01/2021,TX,47.0,47.0,NaN,F,NaN,"Injection site swelling, redness, warm to the ...",NaN,NaN,...,NaN,1,IM,LA,COVID19 (COVID19 (MODERNA)),Injection site erythema,Injection site pruritus,Injection site swelling,Injection site warmth,NaN
916605,01/01/2021,TX,40.0,40.0,NaN,M,NaN,"Adverse Events: Inflammation in the eye, confu...",NaN,NaN,...,276563,1,SYR,LA,INFLUENZA (SEASONAL) (FLUCELVAX QUADRIVALENT),Chills,Confusional state,Eye inflammation,Headache,Laboratory test
916605,01/01/2021,TX,40.0,40.0,NaN,M,NaN,"Adverse Events: Inflammation in the eye, confu...",NaN,NaN,...,276563,1,SYR,LA,INFLUENZA (SEASONAL) (FLUCELVAX QUADRIVALENT),Pyrexia,White blood cell count decreased,NaN,NaN,NaN
916606,01/01/2021,NV,44.0,44.0,NaN,F,NaN,patient called back the next day and stated he...,NaN,NaN,...,011J20A,1,IM,LA,COVID19 (COVID19 (MODERNA)),Pharyngeal swelling,NaN,NaN,NaN,NaN
916607,01/01/2021,KS,50.0,50.0,NaN,M,NaN,SEVERE chills approximately 13-14 hours after ...,NaN,NaN,...,NaN,UNK,IM,LA,COVID19 (COVID19 (MODERNA)),Abdominal pain,Chills,Sleep disorder,NaN,NaN


In [ ]:
# prompt: what are the values of DIED

df_combine_2021['DIED'].value_counts()


DIED
Y    20392
Name: count, dtype: int64

In [ ]:
# prompt: top 12 value counts

df_symptoms['SYMPTOM1'].value_counts().head(12)


SYMPTOM1
Chills                          56844
Arthralgia                      43081
Dizziness                       32838
Fatigue                         29682
COVID-19                        29674
Headache                        27503
Asthenia                        22798
Pyrexia                         21639
Injection site erythema         18272
Pain                            16171
Pain in extremity               16082
Expired product administered    15008
Name: count, dtype: int64

In [ ]:
df_symptoms['SYMPTOM1'].value_counts().head(12)

SYMPTOM1
No adverse event           7112
Chills                     6241
Injection site erythema    5161
Erythema                   4447
Arthralgia                 3266
Herpes zoster              3026
Dizziness                  2620
Pyrexia                    2616
Injection site pain        2581
Fatigue                    2418
Asthenia                   2300
Headache                   2239
Name: count, dtype: int64

In [ ]:
# prompt: WHAT ARE TOP 10 MOST COMMON VALUE IN SYMPTOM1
df_symptoms['SYMPTOM2'].value_counts().head(12)


SYMPTOM2
Product storage error      4849
Injection site pain        3723
Headache                   3505
No adverse event           3231
Fatigue                    3102
Pain                       2966
Pyrexia                    2656
Pain in extremity          2617
Chills                     2539
Injection site erythema    2241
Injection site swelling    2006
Rash                       1809
Name: count, dtype: int64

In [ ]:
df_symptoms.columns

Index(['VAERS_ID', 'SYMPTOM1', 'SYMPTOM2', 'SYMPTOM3', 'SYMPTOM4', 'SYMPTOM5'], dtype='object')

In [ ]:
# prompt: combine SYMPTOM1,SYMPTOM2 in one column

df_symptoms['SYMPTOMS'] = df_symptoms['SYMPTOM1'].fillna('') + ',' + df_symptoms['SYMPTOM2'].fillna('')


In [ ]:
print(df_symptoms.columns)



Index(['RECVDATE', 'STATE', 'AGE_YRS', 'CAGE_YR', 'CAGE_MO', 'SEX', 'RPT_DATE',
       'SYMPTOM_TEXT', 'DIED', 'DATEDIED', 'L_THREAT', 'ER_VISIT', 'HOSPITAL',
       'HOSPDAYS', 'X_STAY', 'DISABLE', 'RECOVD', 'VAX_DATE', 'ONSET_DATE',
       'NUMDAYS', 'LAB_DATA', 'V_ADMINBY', 'V_FUNDBY', 'OTHER_MEDS', 'CUR_ILL',
       'HISTORY', 'PRIOR_VAX', 'SPLTTYPE', 'FORM_VERS', 'TODAYS_DATE',
       'BIRTH_DEFECT', 'OFC_VISIT', 'ER_ED_VISIT', 'ALLERGIES', 'VAX_TYPE',
       'VAX_MANU', 'VAX_LOT', 'VAX_DOSE_SERIES', 'VAX_ROUTE', 'VAX_SITE',
       'VAX_NAME', 'SYMPTOM1', 'SYMPTOM2', 'SYMPTOM3', 'SYMPTOM4', 'SYMPTOM5',
       'SYMPTOM', 'SYMPTOMS'],
      dtype='object')


In [ ]:
df_symptoms.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1070234 entries, 916600 to 2594447
Data columns (total 48 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   RECVDATE         1070234 non-null  object 
 1   STATE            923390 non-null   object 
 2   AGE_YRS          975545 non-null   float64
 3   CAGE_YR          881058 non-null   float64
 4   CAGE_MO          8245 non-null     float64
 5   SEX              1070234 non-null  object 
 6   RPT_DATE         1293 non-null     object 
 7   SYMPTOM_TEXT     1069604 non-null  object 
 8   DIED             20392 non-null    object 
 9   DATEDIED         18847 non-null    object 
 10  L_THREAT         26261 non-null    object 
 11  ER_VISIT         192 non-null      object 
 12  HOSPITAL         112784 non-null   object 
 13  HOSPDAYS         80015 non-null    float64
 14  X_STAY           771 non-null      object 
 15  DISABLE          27298 non-null    object 
 16  RECOVD           9

In [ ]:
# prompt: how many people DIED by sex

died_by_sex = df_combine_2021.groupby('SEX')['DIED'].value_counts()
print(died_by_sex)


SEX  DIED
F    Y        8105
M    Y       11743
U    Y         544
Name: count, dtype: int64


In [ ]:
# prompt: count SYMPTOM1

countSymtoms1=df_combine_2021['SYMPTOM1'].value_counts().head(10)
print(countSymtoms1)


SYMPTOM1
Chills                     56844
Arthralgia                 43081
Dizziness                  32838
Fatigue                    29682
COVID-19                   29674
Headache                   27503
Asthenia                   22798
Pyrexia                    21639
Injection site erythema    18272
Pain                       16171
Name: count, dtype: int64


In [ ]:
# prompt: what us the most common symtom vaccine manu

most_common_symptom = df_combine_2021['SYMPTOM1'].mode()[0]
print(f"The most common symptom is: {most_common_symptom}")


The most common symptom is: Chills
